In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
import numpy as np
from lhapdf import mkPDF, setVerbosity

tf.keras.backend.clear_session()
setVerbosity(0)

In [ ]:
pdf_set = "NNPDF40_nnlo_as_01180"
pdf_target = mkPDF(pdf_set)


def pid_to_latex(pid):
    """Function to translate the PID number into latex values, useful for plotting"""
    translate = {21: "g", 1: "d", 2: "u", 3: "s", 4: "c", 5: "b", 6: "t"}
    flav = translate[abs(pid)]
    if pid < 0:
        flav = rf"\bar{{{flav}}}"
    return flav

In [ ]:
q0 = 1.65  # Reference scale
npoints = int(5e4)  # How many points to use for training
xgrid = np.concatenate([np.logspace(-5, -1, npoints // 2), np.linspace(0.1, 1, npoints // 2)])
pdf_grid_all = pdf_target.xfxQ2(xgrid, np.ones_like(xgrid) * q0**2)

In [ ]:
def generate_model(outputs=1, input_layer=None, nlayers=3, units=14, activation="tanh"):
    """
    Create a tensorflow sequential model where all intermediate layers have the same size
    This function accepts an already constructed layer as the input.

    All hidden layers will have the same number of nodes for simplicity

    Arguments:
        outputs: int (default=1)
            number of output nodes (how many flavours are we training)
        input_layer: KerasTensor (default=None)
            if given, sets the input layer of the sequential model
        nlayers: int
            number of hidden layers of the network
        units: int
            number of nodes of every hidden layer in the network
        activation: str
            activation function to be used by the hidden layers (ex: 'tanh', 'sigmoid', 'linear')
    """
    model = Sequential(name="pdf")
    if input_layer is not None:
        model.add(input_layer)
    for _ in range(nlayers):
        model.add(keras.layers.Dense(units, activation=activation))
    model.add(keras.layers.Dense(outputs, activation="linear"))

    opt = keras.optimizers.Nadam()
    model.compile(opt, loss="mse")
    return model


pdf_model = generate_model()

In [ ]:
# NOTE: for now compute only for the gluon. NTK with more outputs simply means a higher dimentionsal NTK
target_pid = 21
name = pid_to_latex(target_pid)

parton_data = np.array([pdf_grid[target_pid] for pdf_grid in pdf_grid_all])[::200]

# The tensorflow model expects the input to be (ndim, batch_size) in our case ndim = 1 (x)
input_xgrid = xgrid[::200].reshape(-1,1)

In [ ]:
def compute_ntk(model, input):
    grad = []
    for x in tf.convert_to_tensor(input):
        with tf.GradientTape() as tape:
            x = tf.reshape(x, shape=(-1,1))
            tape.watch(x)
            pred = model(x)

        # compute gradients df(x)/dtheta
        g = tape.gradient(pred, model.trainable_variables)
        # concatenate the gradients of all trainable variables,
        # not discriminating between weights and biases
        g = tf.concat([tf.reshape(i, shape=(-1,1)) for i in g], axis=0)
        grad.append(g)

    grad = tf.concat(grad,axis=1)
    ntk = tf.einsum('ij,ik->jk',grad,grad)
    return ntk


In [ ]:
ntks = []
for epochs in 10*[2]:
    pdf_model.fit(input_xgrid, parton_data, epochs=epochs, validation_split=0.3)
    ntks.append(compute_ntk(pdf_model,input_xgrid))